In [1]:
%matplotlib inline
from datetime import datetime
import os

from model import get_1000fps_model
from data import dataGenerator, load_dataset

import numpy as np
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras import backend

import cv2
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def extract_contour_mask(mask):
    kernel = np.ones((3,3), np.uint8)
    erosion = cv2.erode(mask, kernel, iterations =1)
    return mask - erosion

def save_test_images(pred,images):
    pred = (pred < 0.5).astype(np.uint8)
    
    results = []
    for i in range(pred.shape[0]):
        mask = extract_contour_mask(pred[i])
        mask = np.stack([mask*255,mask*0,mask*0],axis=-1)
        mask = cv2.resize(mask,(256,256))
        results.append(cv2.add(images[i],mask))
    results = np.concatenate(results)
    
    results = cv2.cvtColor(results,cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(model_dir,"example_output.png"),results)

In [3]:
# set-up 
input_size = (96,96)
example_dir = "../example/"
model_dir = os.path.join("../save/",datetime.now().strftime("basic-%m%d-%H%M"))
os.makedirs(model_dir,exist_ok=True)

In [4]:
## load example images
images = []
inputs = []
for filename in os.listdir(example_dir):
    ex_path = os.path.join(example_dir,filename)
    image = cv2.imread(ex_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    input_ = cv2.resize(image,input_size)
    input_ = input_/255.
    image = cv2.resize(image,(256,256))
    inputs.append(input_)
    images.append(image)

inputs = np.stack(inputs,axis=0)

In [5]:
## load data generator
batch_size = 64

dataset = load_dataset(input_size=input_size)
trainset, testset = train_test_split(dataset,test_size=0.1)

train_nums = len(trainset)
train_steps = train_nums // batch_size

test_nums = len(testset)
test_steps = test_nums // batch_size

traingen = dataGenerator(trainset, input_size, batch_size)
testgen = dataGenerator(testset, input_size, batch_size)

In [ ]:
# set-up model
model = get_1000fps_model(input_size)
model.compile("adam",
              loss="mse",
             metrics=['mse'])

In [ ]:
hist = model.fit_generator(traingen,
                           train_steps,
                           epochs=50,
                           verbose=0, 
                           validation_data = testgen,
                           validation_steps = test_steps,
                           callbacks=[TQDMNotebookCallback()])

pred = model.predict_on_batch(inputs)
save_test_images(pred,images)

# Draw the results
train_mse = hist.history['mean_squared_error']
valid_mse = hist.history['val_mean_squared_error']
df = pd.DataFrame({"mse-train":train_mse,"mse-valid":valid_mse})
df.plot().figure.savefig(os.path.join(model_dir,"training.png"))
    
# Save Model
# serialize model to JSON
model_json = model.to_json()
with open(os.path.join(model_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(model_dir,"model.h5"))
print("Saved model to disk")